# How to use [ReMatching](https://arxiv.org/abs/2305.09274]) to compute a functional map?

In [1]:
import os

import numpy as np

from geomfum.refine import ZoomOut
from geomfum.shape import TriangleMesh
from geomfum.shape.hierarchical import HierarchicalMesh

DATA_DIR = os.environ["GEOMFUM_DATA_DIR"]

[Load meshes](load_mesh_from_file.ipynb).

In [2]:
filename_a = "cat-00.off"
filename_b = "lion-00.off"

mesh_a = TriangleMesh.from_file(f"{DATA_DIR}/{filename_a}")
mesh_b = TriangleMesh.from_file(f"{DATA_DIR}/{filename_b}")

(mesh_a.n_vertices, mesh_a.n_faces), (mesh_b.n_vertices, mesh_b.n_faces)

((7207, 14410), (5000, 9996))

Create [hierarchical meshes](./hierarchical_mesh.ipynb).

In [3]:
hmesh_a = HierarchicalMesh.from_registry(mesh_a, min_n_samples=1000)
hmesh_b = HierarchicalMesh.from_registry(mesh_b, min_n_samples=1000)

(
    (hmesh_a.low.n_vertices, hmesh_a.low.n_faces),
    (hmesh_b.low.n_vertices, hmesh_b.low.n_faces),
)

((1004, 2004), (1021, 2038))

[Set Laplace eigenbasis](./mesh_laplacian_spectrum.ipynb) for each low-resolution mesh.

In [4]:
hmesh_a.low.find_laplacian_spectrum(spectrum_size=10, set_as_basis=True)
hmesh_b.low.find_laplacian_spectrum(spectrum_size=10, set_as_basis=True)

hmesh_a.low.basis.use_k = 6
hmesh_b.low.basis.use_k = 5

Extend the basis for the high-resolution meshes.

In [5]:
hmesh_a.extend_basis(set_as_basis=True)
hmesh_b.extend_basis(set_as_basis=True)

(
    hmesh_a.high.basis.vecs.shape,
    hmesh_b.high.basis.vecs.shape,
)

((7207, 10), (5000, 10))

Assume we have a [valid functional map](./functional_map.ipynb) $C$ between `hmesh_a.low` and `hmesh_b.low` (which for demonstration purposes, we instantiate randomly).

In [6]:
fmap_matrix = np.random.uniform(
    size=(
        hmesh_b.low.basis.spectrum_size,
        hmesh_a.low.basis.spectrum_size,
    )
)

fmap_matrix.shape

(5, 6)

Now, this functional map can be seamlessly used with the high-resolution meshes. For example, we can [upsample it with ZoomOut](./refine_functional_map.ipynb).

In [7]:
upsampler = ZoomOut(nit=2, step=(2, 1))

upsampled_fmap_matrix = upsampler(
    fmap_matrix,
    hmesh_a.high.basis,
    hmesh_b.high.basis,
)

upsampled_fmap_matrix.shape

(7, 10)

NB: `mesh_a` and `hmesh_a.high` are the same object, so it is indiferent which one to use.

## Further reading

* [How to compute a functional map?](./functional_map.ipynb)